In [3]:
Satwik Pani%matplotlib widget
from jax.config import config
config.update("jax_enable_x64", True)

SyntaxError: invalid syntax (785382827.py, line 1)

In [4]:
Satwik Paniimport numpy as np
import magpylib as magpy
import matplotlib.pyplot as plt 
#from scipy.special import ellipk, ellipe, elliprf, elliprj 
from scipy.integrate import quad
from scipy.spatial.transform import Rotation 
import scipy.optimize
import pandas as pd
import sklearn
from scipy.interpolate import RBFInterpolator
from scipy.interpolate import LinearNDInterpolator 
 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import einops as eo
import joblib
import dill 
import time
from functools import partial
import jax.numpy as jnp
import jax.scipy as jsp
import jax
import sys


x = jax.random.uniform(jax.random.PRNGKey(0), (1000,), dtype=jnp.float64)
x.dtype # --> dtype('float64')


SyntaxError: invalid syntax (1398662939.py, line 1)

In [ ]:
Satwik Panidef ellipp(n, m):
    n, m = (np.asarray(x) for x in (n, m))
    if np.any(m >= 1):
        raise ValueError("m must be < 1")
    y = 1 - m
    rf = elliprf(0, y, 1)
    rj = elliprj(0, y, 1, 1 - n)
    return rf + rj * n / 3

@jax.jit
#assumes p is positive
def ellipc(kc, p, c, s):
    kc = jnp.asarray(kc, dtype=jnp.float64)
    p = jnp.asarray(p, dtype=jnp.float64)
    c = jnp.asarray(c, dtype=jnp.float64)
    s = jnp.asarray(s, dtype=jnp.float64)
    #print(p[p<=0])
    #if jnp.any(jnp.equal(kc,0)):
    #    raise ValueError("kc cannot be 0.")
    errtol=1e-10
    k = jnp.abs(kc)

    pp = p.copy()
    cc = c.copy()
    ss = s.copy() 
    em = 1.
    cond = p>0 

    #pp = pp.at[cond].set(jnp.sqrt(p[cond]))
    #ss = ss.at[cond].set((s/pp)[cond])
    pp = jnp.where(cond, jnp.sqrt(p), pp)
    ss = jnp.where(cond, s/pp, ss)
    #if not jnp.min(p, axis=None)>0:
    #    f = kc[~cond]**2 
    #    q = 1 - f
    #    g = 1 - pp[~cond]
    #    f = f - pp[~cond]
    #    q = q*(ss[~cond]-c[~cond]*pp[~cond])
    #    #pp = pp.at[~cond].set(np.sqrt(f/g))
    #    pp = jnp.where(~cond, jnp.sqrt(f/g), pp)
    #    cc = (c[~cond]-ss[~cond])/g
    #    #ss = ss.at[~cond].set(-q/(pp[~cond]*g**2+cc*pp[~cond]))
    #    ss = jnp.where(~cond, -q/(pp[~cond]*g**2+cc*pp[~cond]), ss)
    f = cc.copy()
    cc = cc + ss/pp
    g = k/pp
    ss = 2*(ss+f*g)
    pp = g + pp
    g = em
    em = k + em
    kk = k.copy()
    #while(jnp.max(jnp.abs((g-k)/g))>errtol):
    for i in range(10):
        k = 2*jnp.sqrt(kk)
        kk = k*em
        f = cc.copy()
        cc = cc + ss/pp
        g = kk/pp
        ss = 2*(ss+f*g)
        pp = g + pp
        g = em.copy()
        em = k + em
    return (jnp.pi/2)*(ss+cc*em)/(em*(em+pp))

def ellipc_np(kc, p, c, s):
    kc, p, c, s = (np.asarray(x, dtype = np.float64) for x in (kc, p, c, s))
    #print(p[p<=0])
    if np.any(kc==0):
        raise ValueError("kc cannot be 0.")
    errtol=1e-10
    k = np.abs(kc)

    pp = p.copy()
    cc = c.copy()
    ss = s.copy() 
    em = 1.
    cond = p>0 

    pp[cond] = np.sqrt(p[cond])
    ss[cond] = (s/pp)[cond]

    if np.any(~cond):
        f = kc[~cond]**2 
        q = 1 - f
        g = 1 - pp[~cond]
        f = f - pp[~cond]
        q = q*(ss[~cond]-c[~cond]*pp[~cond])
        pp[~cond] = np.sqrt(f/g)
        cc = (c[~cond]-ss[~cond])/g
        ss[~cond] = (-q/(pp[~cond]*g**2+cc*pp[~cond]))
    f = cc.copy()
    cc = cc + ss/pp
    g = k/pp
    ss = 2*(ss+f*g)
    pp = g + pp
    g = em
    em = k + em
    kk = k.copy()
    while(np.any(np.abs(g-k)>g*errtol)):
        k = 2*np.sqrt(kk)
        kk = k*em
        f = cc.copy()
        cc = cc + ss/pp
        g = kk/pp
        ss = 2*(ss+f*g)
        pp = g + pp
        g = em.copy()
        em = k + em
    return (np.pi/2)*(ss+cc*em)/(em*(em+pp))
 
def test_ellipc(n, k):
    m = k**2
    kc = np.sqrt(1-m)
    elliptic = np.array([ellipk(m), ellipe(m), ellipp(n, m)])
    arg1 = np.array([kc, kc, kc])
    arg2 = np.array([1, 1, -n+1])
    arg3 = np.array([1, 1, 1])
    arg4 = np.array([1, kc**2, 1])

    elliptic_results = ellipc(arg1, arg2, arg3, arg4)
    test_1 = np.allclose(elliptic, elliptic_results, 1e-7) 
    return test_1
    
@jax.jit
def alpha(xi, rr, R):
    return R/(jnp.sqrt(xi**2 +(rr+R)**2 ))
@jax.jit
def beta(xi, rr, R):
    return xi/(jnp.sqrt(xi**2 +(rr+R)**2 ))

@jax.jit
def k(xi, rr, R):
    num = (xi**2 + (R-rr)**2)
    den = (xi**2 + (R+rr)**2)
    return jnp.sqrt(num/den)

def Br_func(rr, zz, R=0.03, L=0.02, C=1.4):
    Bmax = C
    M = (Bmax)*(np.sqrt(4*L**2 +R**2)/(2*L))

    xi_p, xi_m = zz+L, zz-L
    alpha_p, alpha_m = alpha(xi_p, rr, R), alpha(xi_m, rr, R)


    kp, km = k(xi_p, rr, R), k(xi_m, rr, R)

    Br = (M*R/np.pi)* (alpha_p*ellipc(kp, 1, 1, -1) - alpha_m*ellipc(km, 1, 1, -1))
    return Br

def Bz_polar_func(rr, zz, R=0.03, L=0.02, C=1.4):
    Bmax = C
    M = (Bmax)*(np.sqrt(4*L**2 +R**2)/(2*L))
    xi_p, xi_m = zz+L, zz-L
    beta_p, beta_m = beta(xi_p, rr, R), beta(xi_m, rr, R)
    gamma = (R-rr)/(R+rr)

    kp, km = k(xi_p, rr, R), k(xi_m, rr, R)

    Bz = (R*M/(np.pi*(rr+R))) * (beta_p*ellipc(kp, gamma**2, 1, gamma) - beta_m*ellipc(km, gamma**2, 1, gamma))
    return Bz


def Bxyz_func(xx, yy, zz, R=3, L=2, C=1e3):
    tt = np.arctan2(yy,xx)
    rr = np.sqrt(xx**2 + yy**2)
    Br = Br_func(rr, zz, R, L, C)
    Bx = Br * np.cos(tt)
    By = Br * np.sin(tt)
    Bz = Bz_polar_func(rr, zz, R, L, C)
    return Bx, By, Bz

#def Bx_func(xx, yy, zz, R=3, L=2, C=1e3):
#    tt = np.arctan2(yy,xx)
#    rr = np.sqrt(xx**2 + yy**2)
#    Br = Br_func(rr, zz, R, L, C)
#    Bx = Br * np.cos(tt)
#    return Bx

#def By_func(xx, yy, zz, R=3, L=2, C=1e3):
#    tt = np.arctan2(yy,xx)
#    rr = np.sqrt(xx**2 + yy**2)
#    Br = Br_func(rr, zz, R, L, C)
#    By = Br * np.sin(tt)
#    return By

#def Bz_func(xx, yy, zz, R=3, L=2, C=1e3):
#    rr = np.sqrt(xx**2 + yy**2)
#    Bz = Bz_polar_func(rr, zz, R, L, C)
#    return Bz

def Bx_func(xx, yy, zz, R=3, L=2, C=1e3):
    return Bxyz_func(xx, yy, zz, R, L, C)[0]

def By_func(xx, yy, zz, R=3, L=2, C=1e3):
    return Bxyz_func(xx, yy, zz, R, L, C)[1]

def Bz_func(xx, yy, zz, R=3, L=2, C=1e3):
    return Bxyz_func(xx, yy, zz, R, L, C)[2]

def Bx(X, x0, y0, z0, rx, ry, rz, R, L, C):
    rot_vec = np.array([rx, ry, rz])
    r = Rotation.from_rotvec(rot_vec)
    Xrot = r.apply(X.T).T
    x, y, z = Xrot
    xin, yin, zin = x-x0, y-y0, z-z0

    Bx = Bx_func(xin, yin, zin, R, L, C)
    return Bx


def By(X, x0, y0, z0, rx, ry, rz, R, L, C):
    rot_vec = np.array([rx, ry, rz])
    r = Rotation.from_rotvec(rot_vec)
    Xrot = r.apply(X.T).T
    x, y, z = Xrot
    xin, yin, zin = x-x0, y-y0, z-z0

    By = By_func(xin, yin, zin, R, L, C)
    return By

def Bz(X, x0, y0, z0, rx, ry, rz, R, L, C):
    rot_vec = np.array([rx, ry, rz])
    r = Rotation.from_rotvec(rot_vec)
    Xrot = r.apply(X.T).T
    x, y, z = Xrot
    xin, yin, zin = x-x0, y-y0, z-z0

    Bz = Bz_func(xin, yin, zin, R, L, C)
    return Bz


def Bx_no_rot(x0, y0, z0, R, L, X, C):
    x, y, z = X
    xin, yin, zin = x-x0, y-y0, z-z0

    Bx = Bx_func(xin, yin, zin, R, L, C)
    return Bx


def By_no_rot(x0, y0, z0, R, L, X, C):
    x, y, z = X
    xin, yin, zin = x-x0, y-y0, z-z0

    By = By_func(xin, yin, zin, R, L, C)
    return By

def Bz_no_rot(x0, y0, z0, R, L, X, C):
    x, y, z = X
    xin, yin, zin = x-x0, y-y0, z-z0

    Bz = Bz_func(xin, yin, zin, R, L, C)
    return Bz

In [ ]:
Satwik Panifolder = "data/PIM_03202023"
file_name = "DataAvg.txt"
file = f"{folder}/{file_name}"

shape_points = {"i":7*10+1,"j": 7*10+1,"k": 6*5+2}

df = pd.read_csv(file).sort_values(["z", "y", "x"])

X = np.array(df.loc[:, "x":"z"]) # unpacks to x, y, z
M = np.array(df.loc[:, "Mx":"Mz"])
df


In [ ]:
Satwik Paniimport numpy as np
import magpylib as magpy
import pyvista as pv

pv.set_jupyter_backend('client') # improve rending in a jupyter notebook

coil1 = magpy.Collection()
#for z in np.linspace(-8, 8, 16):
#    winding = magpy.current.Loop(
#        current=100,
#        diameter=2,
#        position=(0,0,z),
#    )
#    coil1.add(winding)


grid = pv.UniformGrid(
    dimensions=list(shape_points.values()),
    spacing=(0.2, 0.2, 0.2),
    origin=(-5, -5, 0),
)

# compute B-field and add as data to grid
#grid['B'] = coil1.getB(grid.points)
grid['B'] = M
print(grid['B'].shape)

# compute field lines
seed = pv.Disc(inner=0.2, outer=2, r_res=10, c_res=63)
strl = grid.streamlines_from_source(
    seed,
    vectors='B',
    max_time=180,
    initial_step_length=0.01,
    integration_direction='both',
    progress_bar=True
)

# create plotting scene
pl = pv.Plotter()

# add field lines and legend to scene
legend_args = {
    'title': 'B [mT]',
    'title_font_size': 20,
    'color': 'black',
    'position_y': 0.25,
    'vertical': True,
}

# draw coils
# magpy.show(coil1, canvas=pl, backend='pyvista')

# add streamlines
pl.add_mesh(
    strl.tube(radius=.02),
    cmap="RdBu",
    scalar_bar_args=legend_args,
)
# display scene
pl.camera.position=(160, 10, -10)
pl.set_background("white")
pl.show()

In [ ]:
Satwik Pani#
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import magpylib as magpy
from matplotlib.widgets import Slider, Button
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, interactive_output, HBox, VBox
from IPython.display import display

df_xy = (df[df["z"]==0]) 



M_magnitude = np.linalg.norm(eo.rearrange(M,"(i j k) c -> i j k c", **shape_points), axis=2)

cmap = plt.cm.viridis
norm = plt.Normalize(M_magnitude.min(), M_magnitude.max())

X_z = eo.rearrange(np.array(df_xy.loc[:, "x":"z"]), "(i j) c -> i j c", i=shape_points["i"], j=shape_points["j"])


M_z = eo.rearrange(np.array(df_xy.loc[:, "Mx":"Mz"]), "(i j) c -> i j c", i=shape_points["i"], j=shape_points["j"])
    
x = X_z[:, :, 0]
y = X_z[:, :, 1]

u = M_z[:, :, 1]
v = -M_z[:, :, 0]

In [ ]:
Satwik Panifig, ax = plt.subplots(figsize=(4, 4))
#fig.colorbar()

fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax)
#plt.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap))
#@interact(z=FloatSlider(min=0, max=5, step=0.2, value=1.2, description='z (mm)'))
def plot_streamlines(z):
    df_xy = df[df["z"]==z] 
    X_z = eo.rearrange(np.array(df_xy.loc[:, "x":"z"]), "(i j) c -> i j c", i=shape_points["i"], j=shape_points["j"]) # unpacks to x, y, z
    M_z = eo.rearrange(np.array(df_xy.loc[:, "Mx":"Mz"]), "(i j) c -> i j c", i=shape_points["j"], j=shape_points["i"])
    

    
    x = X_z[:, :, 0]
    y = X_z[:, :, 1]

    u = M_z[:, :, 1]
    v = -M_z[:, :, 0]

    Mz_magnitude = np.linalg.norm(M_z, axis=2)
    
   
    cmap = plt.cm.viridis
    norm = plt.Normalize(Mz_magnitude.min(), Mz_magnitude.max())
    
    
    ax.clear()
    ax.streamplot(x, y, u, v, density=0.5, linewidth=1, color=Mz_magnitude, cmap=cmap)
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.grid()
    ax.set_title(f"2D Streamline Plot (z: {z} mm)")
    # Update the plot
    fig.canvas.draw_idle()
plot_streamlines(1)

    

In [ ]:
Satwik Panifig, ax = plt.subplots(figsize=(4, 4))
plot_streamlines(3)


In [ ]:
import scipy as sp
sensor = magpy.Sensor(position=(0,0,0))
magnet = magpy.magnet.Cylinder(magnetization=(0, 0, 1210), dimension=(5, 2))




folder = "data/PIM_03202023"
file_name = "DataAvg.txt"
file = f"{folder}/{file_name}"

shape_points = {"i":7*10+1,"j": 7*10+1,"k": 6*5+2}

df = pd.read_csv(file).sort_values(["z", "y", "x"])


X = np.array(df.loc[:, "x":"z"]) 
M = np.array(df.loc[:, "Mx":"Mz"])


sensor.position = X

def getField(x, sensor, magnet, dx=0.0001):
    xdx = x + dx
    magnet.magnetization=(0,0,x[5])
    magnet.position = x[:3]
    phi = x[3]
    theta = x[4]
    rot = [np.cos(theta/2), np.sin(theta/2)*np.cos(phi), np.sin(theta/2)*np.sin(phi), 0]
    magnet.orientation = sp.spatial.transform.Rotation.from_quat(rot)

    f = magpy.getB(sources=magnet, observers=sensor)
    #xdx = x + dx
    return f
    

def cost(x, B, sensor, magnet):
    diff = getField(x, sensor, magnet)-B
    return np.sum((diff)**2)

guess = np.array([-5,-2,30, np.pi/8, np.pi/4, 1400])

#sol = scipy.optimize.least_squares(cost, x0=guess, args=(M, sensor, magnet), bounds=([-7,-7,30,-np.pi,-np.pi, 1000],[7,7,50,np.pi, np.pi, 2000]))
#sol.x, sol.success


In [ ]:
#sol.x, sol.success, sol.message

In [ ]:
#360*sol.x[3:5]/(2*np.pi)

In [ ]:
#%timeit getField(guess, sensor, magnet)

In [5]:
def C(kc, p, c, s):
    return c*sp.special.elliprf(0,kc**2, 1)+ (1/3)*(c-p*s)*sp.special.elliprj(0, kc**2, 1, p)
%timeit C(0.1, 0.2, 0.3, 0.4)

NameError: name 'sp' is not defined

In [ ]:
 #rot_vec = theta*np.array([np.cos(phi), np.sin(phi), 0])
  #  r = Rotation.from_rotvec(rot_vec)
   # Xrot = r.apply(X.T).T
    #x, y, z = Xrot
    #xin, yin, zin = x-x0, y-y0, z-z0
def B_np(position, R=0.03, L=0.02, Bmax=1.4):
    C=ellipc_np
    xx, yy, zz = position.T
    tt = np.arctan2(yy,xx)
    rr = np.sqrt(xx**2 + yy**2)
    M = (Bmax)*(np.sqrt(4*L**2 +R**2)/(2*L))

    xi_p, xi_m = zz+L, zz-L
    alpha_p, alpha_m = alpha(xi_p, rr, R), alpha(xi_m, rr, R)
    beta_p, beta_m = beta(xi_p, rr, R), beta(xi_m, rr, R)
    gamma = (R-rr)/(R+rr)


    kp, km = k(xi_p, rr, R), k(xi_m, rr, R)

    Br = (M*R/np.pi)* (alpha_p*C(kp, 1, 1, -1) - alpha_m*C(km, 1, 1, -1))
    Bx = Br * np.cos(tt)
    By = Br * np.sin(tt)
    Bz = (R*M/(np.pi*(rr+R))) * (beta_p*C(kp, gamma**2, 1, gamma) - beta_m*C(km, gamma**2, 1, gamma))

    return Br, Bz

@jax.jit
def B_jnp(position, R=0.025, L=0.02, Bmax=1.4):
    C=ellipc
    xx, yy, zz = jnp.asarray(position).T
    tt = jnp.arctan2(yy,xx)
    rr = jnp.sqrt(xx**2 + yy**2)
    M = (Bmax)*(jnp.sqrt(4*L**2 +R**2)/(2*L))

    xi_p, xi_m = zz+L, zz-L
    alpha_p, alpha_m = alpha(xi_p, rr, R), alpha(xi_m, rr, R)
    beta_p, beta_m = beta(xi_p, rr, R), beta(xi_m, rr, R)
    gamma = (R-rr)/(R+rr)


    kp, km = k(xi_p, rr, R), k(xi_m, rr, R)

    Br = (M*R/jnp.pi)* (alpha_p*C(kp, 1, 1, -1) - alpha_m*C(km, 1, 1, -1))
    Bx = Br * jnp.cos(tt)
    By = Br * jnp.sin(tt)
    Bz = (R*M/(jnp.pi*(rr+R))) * (beta_p*C(kp, gamma**2, 1, gamma) - beta_m*C(km, gamma**2, 1, gamma))

    return jnp.array([Bx, By, Bz])

@jax.jit
def B_dipole(position, rotation, M0, shape=(25.4*3/16, 25.4/16)):
    R = jnp.sqrt(jnp.sum(position**2))
    B = (M0*(shape[0]/2)**2 * shape[1] *jnp.pi /( 4*jnp.pi))*((3*position*(jnp.matmul(position, rotation))[:, jnp.newaxis]/R**5)-rotation/R**3)
    return B

@jax.jit
def getField_dipole(x, positions):
    #magnetization=x[5]
    magnetization=1210
    position = x[:3]
    phi = x[3]
    theta = x[4]
    axis = jnp.array([jnp.sin(theta)*jnp.cos(phi), jnp.sin(theta)*jnp.sin(phi), jnp.cos(theta)])
    return B_dipole(positions-position, axis, magnetization)
@jax.jit
def rotation_matrix(axis, theta):
        """
        Return the rotation matrix associated with counterclockwise rotation about
        the given axis by theta radians.
        """
        axis = jnp.asarray(axis)
        axis = axis / jnp.sqrt(jnp.dot(axis, axis))
        a = jnp.cos(theta / 2.0)
        b, c, d = -axis * jnp.sin(theta / 2.0)
        aa, bb, cc, dd = a * a, b * b, c * c, d * d
        bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
        return jnp.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                        [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                        [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])


@jax.jit
def getField_jnp(x, positions):
    magnetization=x[5]
    #magnetization=1200
    position = x[:3]
    phi = x[3]
    theta = x[4]
    axis = jnp.array([jnp.cos(phi), jnp.sin(phi), 0])
    pos_trans = jnp.einsum("i j, k j ->k i" ,rotation_matrix(-axis, -theta), positions)-position
    
    return B_jnp(pos_trans, Bmax=magnetization).T

@jax.jit
def cost_jnp(x, B, positions):            
    diff = getField_jnp(x, positions)-B
    return np.sum((diff)**2)


@jax.jit
def cost_dipole(x, B, positions):
    diff = getField_dipole(x, positions)-B
    return np.sum((diff)**2)


In [ ]:
from jax.scipy import optimize
guess = np.array([-1,-1, 40, np.pi/8, np.pi/4])
#sol = optimize.minimize(cost_jnp, x0=guess, args=(M, X), method="BFGS", tol=1e-6, options={"gtol": 1e-6})

In [ ]:
#sol.x, sol.status, sol.fun

In [ ]:
#360*sol.x[3:5]/(2*np.pi)

In [ ]:
guess = np.array([7,7, 40, jnp.pi/3, jnp.pi/3])
cost_with_grad = jax.value_and_grad(cost_dipole)
bounds=([-7,-7,-100,-4*np.pi,-2*np.pi],[7,7,100,4*np.pi, 2*np.pi])
bounds = list(zip(*bounds))
sol = scipy.optimize.minimize(cost_with_grad, x0=guess, args=(M, X), jac=True, bounds=bounds, tol=1e-1000)
sol.x, sol.success

In [ ]:
sol.x, sol.status, sol.fun, 

In [ ]:
360*sol.x[3:5]/(2*np.pi)

In [ ]:
diff = np.abs(getField_dipole(sol.x, X)-M)
np.max(diff), np.min(diff), np.mean(diff)

In [ ]:
sol = scipy.optimize.minimize(cost_with_grad, x0=guess, args=(M[::], X[::]), jac=True, bounds=bounds, tol=1e-1000)
sol.x, sol.success

In [ ]:
360*sol.x[3:5]/(2*np.pi)